In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
df = pd.read_csv("/content/Clean_train.csv")

In [ ]:
x = df['text']
y = df['target']

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
x_tfidf = tfidf_vectorizer.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42 ,
                                                    shuffle = True)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,
                                  random_state=42)
lgbm_model = LGBMClassifier(n_estimators=100,
                            random_state=42)

In [ ]:
voting_model = VotingClassifier(
    estimators=[('rf', rf_model), ('lgbm', lgbm_model)],
    voting='soft'  # Use can also use hard voting voting = 'hard'
)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Perform k-fold cross-validation
accuracy_scores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_val_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    voting_model.fit(x_train_fold, y_train_fold)

    y_pred_voting = voting_model.predict(x_val_fold)

    accuracy_fold = accuracy_score(y_val_fold, y_pred_voting)
    accuracy_scores.append(accuracy_fold)

[LightGBM] [Info] Number of positive: 2107, number of negative: 2765
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013967 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6625
[LightGBM] [Info] Number of data points in the train set: 4872, number of used features: 443
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.432471 -> initscore=-0.271776
[LightGBM] [Info] Start training from score -0.271776
[LightGBM] [Info] Number of positive: 2055, number of negative: 2817
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6637
[LightGBM] [Info] Number of data points in the train set: 4872, number of used features: 448
[LightGBM] [Info] [bin

In [ ]:
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Mean Accuracy of Voting Classifier with 5-fold cross-validation:", mean_accuracy)

Mean Accuracy of Voting Classifier with 5-fold cross-validation: 0.7786535303776684
